# Preprocess

## Imports

In [32]:
import pandas as pd
from IPython.display import clear_output

In [33]:
%load_ext autoreload
%autoreload 2
clear_output()

In [37]:
from Dude import stop_id_calculator

## Varriables

In [3]:
PATH = "../data/"
rp, stp , sp ,tp = ("routes","stop_times","stops","trips")

In [6]:
#routes = pd.read_csv(f"{PATH}{rp}.txt")
stop_times = pd.read_csv(f"{PATH}{stp}.txt")
#stops = pd.read_csv(f"{PATH}{sp}.txt")
trips = pd.read_csv(f"{PATH}{tp}.txt")

# Workspace

In [5]:
trips

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,boarding_door
0,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0
1,78,B864711,B86471APPMAA-081,"Újpalota, Nyírpalota út",0,B86471_2770_1_0,V014,1.0,2.0,NaN
2,2240,B945231,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_0,MW15,1.0,2.0,NaN
3,2240,B9452310,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_8,MW15,1.0,2.0,NaN
4,2240,B9452311,B94523ASZSzN-041,"Újpalota, Szentmihályi út",0,B94523_2240_50_9,MW15,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
221746,H5,H8719_25,8281,Batthyány tér,1,NaN,NaN,NaN,NaN,NaN
221747,H5,H8720_29,8281,Békásmegyer,0,NaN,NaN,NaN,NaN,NaN
221748,H5,H8729_25,8281,Batthyány tér,1,NaN,NaN,NaN,NaN,NaN
221749,H5,H8730_29,8289,Békásmegyer,0,NaN,NaN,NaN,NaN,NaN


In [13]:
stop_time_trips = pd.merge(trips,stop_times, on="trip_id")
stop_time_trips.head()

,route_id,trip_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,boarding_door,stop_id,arrival_time,departure_time,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0,008569,03:50:00,03:50:00,0,NaN,NaN,1.0,0.0
1,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0,F03291,03:50:00,03:50:00,1,NaN,NaN,NaN,451.0
2,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0,F03403,03:51:00,03:51:00,2,NaN,NaN,NaN,1113.0
3,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0,F03402,03:52:00,03:52:00,3,NaN,NaN,NaN,2171.0
4,78,B864691,B86469AHCMAA-081,"Újpalota, Nyírpalota út",0,B86469_2770_1_0,V014,1.0,2.0,2.0,F03400,03:53:00,03:53:00,4,NaN,NaN,NaN,2637.0


In [31]:
print(
    f"""
    Number of identical routes: {stop_time_trips.route_id.unique().size}
    """
)


    Number of identical routes: 519
    


In [42]:
nodes = stop_time_trips.stop_id.value_counts()